In [41]:
# !conda update -n base -c defaults conda

# !pip install geocoder
print('geocoder has been installed.')

# !pip install geopy
print('geopy has been installed.')

# !pip install folium=0.5.0
print('folium has been installed.')

# !pip install beautifulsoup4
print('beautifulsoup4 has been installed.')

# !pip install lxml
print('lxml has been installed.')

# !pip install html5lib
print('html5lib has been installed.')

# !pip install geopandas
print('geopandas has been installed.')

geocoder has been installed.
geopy has been installed.
folium has been installed.
beautifulsoup4 has been installed.
lxml has been installed.
html5lib has been installed.
geopandas has been installed.


In [42]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib as mp
from bs4 import BeautifulSoup
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import folium # map rendering library
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import csv
import sys
import urllib3
import lxml
import html5lib
import urllib.request
import re
from requests.exceptions import RequestException
from contextlib import closing
import geopandas as gpd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')


Libraries imported.


In [43]:
# Parse the html content
# soup_Paris = BeautifulSoup(url_Paris, "lxml") (works!!)
# print(soup_Paris.prettify()) # print the parsed data of html (works!!)
# list(soup_Paris.children) (works!!)

In [44]:
# import Wikipedia URL for Paris
# Make a GET request to fetch the raw HTML content
url_Paris = requests.get('https://en.wikipedia.org/wiki/Arrondissements_of_Paris').text
# print(url_Paris)

# print(url_Paris.content) (does not work)
# print(url_Paris.status_code)   # Check the http response status code. This should print 200 (does not work)

soup_Paris = BeautifulSoup(url_Paris, "html.parser")
table_Paris = soup_Paris.find('table',{'class':'wikitable'})

# To get the text without the HTML tags, we just use .text:
print(soup_Paris.title.text)


Arrondissements of Paris - Wikipedia


In [45]:
# extracting table from https://en.wikipedia.org/wiki/Arrondissements_of_Paris
totals = table_Paris.find_all('tr')
nrows = len(totals)

# Header of the DataFrame is the first row of data
header_Paris = totals[0].text.split()
# header_Paris

In [46]:
# Creating the Table
records = []
n=1
while n < nrows :
    District = totals[n].text.split('\n')[1]
    Arrondissement = totals[n].text.split('\n')[2]
    Size = totals[n].text.split('\n')[3]
    Population = totals[n].text.split('\n')[5]
    Population_density = totals[n].text.split('\n')[6]
    records.append((District, Arrondissement, Size, Population, Population_density))
    n=n+1

df_Paris=pd.DataFrame(records, columns=['District', 'Arrondissement','Size km²', 'Population', 'Population Density'])

df_Paris.head(20)

,District,Arrondissement,Size km²,Population,Population Density
0,1st (Ie) R,Louvre,1.826 km2 (0.705 sq mi),"17,700","9,693"
1,2nd (IIe) R,Bourse,0.992 km2 (0.383 sq mi),"20,700","20,867"
2,3rd (IIIe) R,Temple,1.171 km2 (0.452 sq mi),"35,100","29,974"
3,4th (IVe) R,Hôtel-de-Ville,1.601 km2 (0.618 sq mi),"28,600","17,864"
4,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),"60,600","23,849"
5,6th (VIe) L,Luxembourg,2.154 km2 (0.832 sq mi),"45,200","20,984"
6,7th (VIIe) L,Palais-Bourbon,4.088 km2 (1.578 sq mi),"55,400","13,552"
7,8th (VIIIe) R,Élysée,3.881 km2 (1.498 sq mi),"38,700","9,972"
8,9th (IXe) R,Opéra,2.179 km2 (0.841 sq mi),"58,500","26,847"
9,10th (Xe) R,Entrepôt,2.892 km2 (1.117 sq mi),"88,800","30,705"


In [47]:
# Splitting the District column: new data frame with split value columns 
new1 = df_Paris["District"].str.split(" ", n = 2, expand = True)
  
# making separate district column from new data frame 
df_Paris["Districts"]= new1[0]

# making separate arrondissement (FR) column from new data frame 
df_Paris["Arrfr"]= new1[1]

# making separate River Bank column from new data frame 
df_Paris["RiverBank"]= new1[2]
  
# Dropping old Name columns 
df_Paris.drop(columns =['District', 'Arrfr'], inplace = True)

#df_Paris.reindex(columns="Districts","Arrondissement","RiverBank","Population","Population Density","Size km²")
# df_Paris.reset_index()
# check dataframe
df_Paris

,Arrondissement,Size km²,Population,Population Density,Districts,RiverBank
0,Louvre,1.826 km2 (0.705 sq mi),"17,700","9,693",1st,R
1,Bourse,0.992 km2 (0.383 sq mi),"20,700","20,867",2nd,R
2,Temple,1.171 km2 (0.452 sq mi),"35,100","29,974",3rd,R
3,Hôtel-de-Ville,1.601 km2 (0.618 sq mi),"28,600","17,864",4th,R
4,Panthéon,2.541 km2 (0.981 sq mi),"60,600","23,849",5th,L
5,Luxembourg,2.154 km2 (0.832 sq mi),"45,200","20,984",6th,L
6,Palais-Bourbon,4.088 km2 (1.578 sq mi),"55,400","13,552",7th,L
7,Élysée,3.881 km2 (1.498 sq mi),"38,700","9,972",8th,R
8,Opéra,2.179 km2 (0.841 sq mi),"58,500","26,847",9th,R
9,Entrepôt,2.892 km2 (1.117 sq mi),"88,800","30,705",10th,R


In [48]:
# splitting the Size column: new data frame with split value columns
new2 = df_Paris["Size km²"].str.split("(", n = 1, expand = True)
# making separate size in km² column from new data frame 
df_Paris["Size km"]= new2[0]

# making separate size in miles² column from new data frame 
df_Paris["Size miles"] = new2[1]

# Dropping old Name columns 
df_Paris.drop(columns = ["Size km²", "Size miles"], inplace = True)


In [49]:
# manually edit Population Density for 11th and 15th districts
df_Paris["Population Density"][11] = 21687
df_Paris["Population Density"][15] = 190544
df_Paris

,Arrondissement,Population,Population Density,Districts,RiverBank,Size km
0,Louvre,"17,700","9,693",1st,R,1.826 km2
1,Bourse,"20,700","20,867",2nd,R,0.992 km2
2,Temple,"35,100","29,974",3rd,R,1.171 km2
3,Hôtel-de-Ville,"28,600","17,864",4th,R,1.601 km2
4,Panthéon,"60,600","23,849",5th,L,2.541 km2
5,Luxembourg,"45,200","20,984",6th,L,2.154 km2
6,Palais-Bourbon,"55,400","13,552",7th,L,4.088 km2
7,Élysée,"38,700","9,972",8th,R,3.881 km2
8,Opéra,"58,500","26,847",9th,R,2.179 km2
9,Entrepôt,"88,800","30,705",10th,R,2.892 km2


In [50]:
# splitting the Size column: new data frame with split value columns (# not working: still showing km² after value)
new3 = df_Paris["Size km"].str.split(" ", n = 1, expand = True)
# making separate size in km² column from new data frame 
df_Paris["Size km²"]= new3[0]

# making separate size in miles² column from new data frame 
df_Paris["to drop"] = new3[1]

# Dropping old Name columns 
df_Paris.drop(columns = ["to drop", "Size km"], inplace = True)
df_Paris

,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²
0,Louvre,"17,700","9,693",1st,R,1.826 km2
1,Bourse,"20,700","20,867",2nd,R,0.992 km2
2,Temple,"35,100","29,974",3rd,R,1.171 km2
3,Hôtel-de-Ville,"28,600","17,864",4th,R,1.601 km2
4,Panthéon,"60,600","23,849",5th,L,2.541 km2
5,Luxembourg,"45,200","20,984",6th,L,2.154 km2
6,Palais-Bourbon,"55,400","13,552",7th,L,4.088 km2
7,Élysée,"38,700","9,972",8th,R,3.881 km2
8,Opéra,"58,500","26,847",9th,R,2.179 km2
9,Entrepôt,"88,800","30,705",10th,R,2.892 km2


In [51]:
# Latitude and Longitude per District
# From https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true
# unable to get it directly from the link
lat_long_Paris = pd.read_csv("arrondissements.csv", sep=';').sort_values('C_AR').reset_index()
lat_long_Paris.drop(columns = ["OBJECTID", "L_AR","N_SQ_AR","index","N_SQ_CO","Geometry"], inplace = True)

lat_long_Paris.columns = ['C_AR', 'C_ARINSEE', 'L_AROFF', 'SURFACE', 'PERIMETRE', 'Geometry', 'LONGUEUR']

In [52]:
# Splitting Geometry into separate columns to obtain latitude and longitude

# splitting the Size column: new data frame with split value columns
new4 = lat_long_Paris["Geometry"].str.split(",", n = 1, expand = True)

# making separate first name column from new data frame
lat_long_Paris["Latitude"]= new4[0]

# making separate last name column from new data frame 
lat_long_Paris["Longitude"] = new4[1]

# Dropping old Name columns 
lat_long_Paris.drop(columns = ["Geometry"], inplace = True)

#lat_long_Paris
#list(lat_long_Paris)

# convert to a Pandas dataframe
lat_long_Paris_df = pd.DataFrame(lat_long_Paris, columns= ['C_AR', 'C_ARINSEE', 'L_AROFF', 'SURFACE', 'PERIMETRE', 'LONGUEUR', 'Latitude', 'Longitude'])
lat_long_Paris_df

,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude
0,1,75101,Louvre,1.824613e+06,6054.936862,6054.680862,48.8625627018,2.33644336205
1,2,75102,Bourse,9.911537e+05,4554.104360,4553.938764,48.8682792225,2.34280254689
2,3,75103,Temple,1.170883e+06,4519.263648,4519.071982,48.86287238,2.3600009859
3,4,75104,Hôtel-de-Ville,1.600586e+06,5420.908434,5420.636779,48.8543414263,2.35762962032
4,5,75105,Panthéon,2.539375e+06,6239.195396,6238.823408,48.8444431505,2.35071460958
5,6,75106,Luxembourg,2.153096e+06,6483.686786,6483.324560,48.8491303586,2.33289799905
6,7,75107,Palais-Bourbon,4.090057e+06,8099.424883,8099.033151,48.8561744288,2.31218769148
7,8,75108,Élysée,3.880036e+06,7880.533268,7880.260661,48.8727208374,2.3125540224
8,9,75109,Opéra,2.178303e+06,6471.588290,6471.389144,48.8771635173,2.33745754348
9,10,75110,Entrepôt,2.891739e+06,6739.375055,6739.168359,48.8761300365,2.36072848785


In [53]:
# combining the two dataframes
paris_et_arrondissements_df = pd.concat([df_Paris, lat_long_Paris_df], axis=1)
lat_long_Paris_df = pd.DataFrame(lat_long_Paris, columns= ['C_AR',  'C_ARINSEE', 'Arrondissement', 'Districts', 'RiverBank', 'Latitude', 'Longitude', 'Population', 'Population Density', 'Size km²', 'PERIMETRE', 'LONGUEUR', 'SURFACE'])
paris_et_arrondissements_df

,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude
0,Louvre,"17,700","9,693",1st,R,1.826 km2,1,75101,Louvre,1.824613e+06,6054.936862,6054.680862,48.8625627018,2.33644336205
1,Bourse,"20,700","20,867",2nd,R,0.992 km2,2,75102,Bourse,9.911537e+05,4554.104360,4553.938764,48.8682792225,2.34280254689
2,Temple,"35,100","29,974",3rd,R,1.171 km2,3,75103,Temple,1.170883e+06,4519.263648,4519.071982,48.86287238,2.3600009859
3,Hôtel-de-Ville,"28,600","17,864",4th,R,1.601 km2,4,75104,Hôtel-de-Ville,1.600586e+06,5420.908434,5420.636779,48.8543414263,2.35762962032
4,Panthéon,"60,600","23,849",5th,L,2.541 km2,5,75105,Panthéon,2.539375e+06,6239.195396,6238.823408,48.8444431505,2.35071460958
5,Luxembourg,"45,200","20,984",6th,L,2.154 km2,6,75106,Luxembourg,2.153096e+06,6483.686786,6483.324560,48.8491303586,2.33289799905
6,Palais-Bourbon,"55,400","13,552",7th,L,4.088 km2,7,75107,Palais-Bourbon,4.090057e+06,8099.424883,8099.033151,48.8561744288,2.31218769148
7,Élysée,"38,700","9,972",8th,R,3.881 km2,8,75108,Élysée,3.880036e+06,7880.533268,7880.260661,48.8727208374,2.3125540224
8,Opéra,"58,500","26,847",9th,R,2.179 km2,9,75109,Opéra,2.178303e+06,6471.588290,6471.389144,48.8771635173,2.33745754348
9,Entrepôt,"88,800","30,705",10th,R,2.892 km2,10,75110,Entrepôt,2.891739e+06,6739.375055,6739.168359,48.8761300365,2.36072848785


In [54]:
df_gpd_arrondissements = gpd.read_file('arrondissements.geojson')
# df_gpd_arrondissements.head()

In [55]:
# Creating map of Paris with its 20 arrondissements (districts)
latitude = 48.8566
longitude= 2.3522

df_gpd_arrondissements = gpd.read_file('arrondissements.geojson')
# df_gpd_arrondissements.head()

# create map of Paris using latitude and longitude values above:
map_Paris = folium.Map(location=[latitude, longitude], tiles='Mapbox Bright', zoom_start=12)

# map_Paris.choropleth(geo_path='arrondissements.geojson')

folium.GeoJson(df_gpd_arrondissements).add_to(map_Paris)

# map_Paris.save(os.path.join('results', 'GeoJSON_and_choropleth_0.html'))

# add markers to map
for lat, lng, label in zip(paris_et_arrondissements_df['Latitude'], paris_et_arrondissements_df['Longitude'], paris_et_arrondissements_df['C_AR']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_Paris)  
    
map_Paris


In [56]:
# temporary stuffs for latter
# get all the links in the page along with its attributes, such as href, title, and its inner Text. (works!!)
for link in soup_Paris.find_all("a"):
    print("Inner Text: {}".format(link.text))
    print("Title: {}".format(link.get("title")))
    print("href: {}".format(link.get("href")))

Inner Text: 
Title: None
href: None
Inner Text: Jump to navigation
Title: None
href: #mw-head
Inner Text: Jump to search
Title: None
href: #p-search
Inner Text: Country
Title: List of sovereign states
href: /wiki/List_of_sovereign_states
Inner Text: France
Title: France
href: /wiki/France
Inner Text: Region
Title: Regions of France
href: /wiki/Regions_of_France
Inner Text: Île-de-France
Title: Île-de-France
href: /wiki/%C3%8Ele-de-France
Inner Text: Department
Title: Departments of France
href: /wiki/Departments_of_France
Inner Text: Paris
Title: Paris
href: /wiki/Paris
Inner Text: Cantons
Title: Cantons of France
href: /wiki/Cantons_of_France
Inner Text: Communes
Title: Communes of France
href: /wiki/Communes_of_France
Inner Text: Préfecture
Title: Prefectures in France
href: /wiki/Prefectures_in_France
Inner Text: Paris
Title: Paris
href: /wiki/Paris
Inner Text: 
Title: Enlarge
href: /wiki/File:Paris_arr_jms-num.gif
Inner Text: Paris
Title: Paris
href: /wiki/Paris
Inner Text: arrondi

In [57]:
paris_arrondissement = ['Paris_01','Paris_02','Paris_03','Paris_04','Paris_05','Paris_06','Paris_07','Paris_081','Paris_09','Paris_10','Paris_11','Paris_12','Paris_13','Paris_14','Paris_15','Paris_16','Paris_17','Paris_18','Paris_19','Paris_20']
# paris_arrondissement

In [58]:
# temporary stuffs for latter (not needed)
# import places of interest per arrondissement (for later)
url_Paris_01 = requests.get('https://en.wikipedia.org/wiki/1st_arrondissement_of_Paris').text
url_Paris_02 = requests.get('https://en.wikipedia.org/wiki/2nd_arrondissement_of_Paris').text
url_Paris_03 = requests.get('https://en.wikipedia.org/wiki/3rd_arrondissement_of_Paris').text
url_Paris_04 = requests.get('https://en.wikipedia.org/wiki/4th_arrondissement_of_Paris').text
url_Paris_05 = requests.get('https://en.wikipedia.org/wiki/5th_arrondissement_of_Paris').text
url_Paris_06 = requests.get('https://en.wikipedia.org/wiki/6th_arrondissement_of_Paris').text
url_Paris_07 = requests.get('https://en.wikipedia.org/wiki/7th_arrondissement_of_Paris').text
url_Paris_08 = requests.get('https://en.wikipedia.org/wiki/8th_arrondissement_of_Paris').text
url_Paris_09 = requests.get('https://en.wikipedia.org/wiki/9th_arrondissement_of_Paris').text
url_Paris_10 = requests.get('https://en.wikipedia.org/wiki/10th_arrondissement_of_Paris').text
url_Paris_11 = requests.get('https://en.wikipedia.org/wiki/11th_arrondissement_of_Paris').text
url_Paris_12 = requests.get('https://en.wikipedia.org/wiki/12th_arrondissement_of_Paris').text
url_Paris_13 = requests.get('https://en.wikipedia.org/wiki/13th_arrondissement_of_Paris').text
url_Paris_14 = requests.get('https://en.wikipedia.org/wiki/14th_arrondissement_of_Paris').text
url_Paris_15 = requests.get('https://en.wikipedia.org/wiki/15th_arrondissement_of_Paris').text
url_Paris_16 = requests.get('https://en.wikipedia.org/wiki/16th_arrondissement_of_Paris').text
url_Paris_17 = requests.get('https://en.wikipedia.org/wiki/17th_arrondissement_of_Paris').text
url_Paris_18 = requests.get('https://en.wikipedia.org/wiki/18th_arrondissement_of_Paris').text
url_Paris_19 = requests.get('https://en.wikipedia.org/wiki/19th_arrondissement').text
url_Paris_20 = requests.get('https://en.wikipedia.org/wiki/20th_arrondissement_of_Paris').text

In [59]:
# temporary stuffs for latter (not needed)
url_Paris = [url_Paris_01, url_Paris_02, url_Paris_03, url_Paris_04, url_Paris_05, url_Paris_06, url_Paris_07, url_Paris_08, url_Paris_09, url_Paris_10, url_Paris_11, url_Paris_12,url_Paris_13,url_Paris_14,url_Paris_15,url_Paris_16,url_Paris_17,url_Paris_18,url_Paris_19,url_Paris_20]
# url_Paris

In [60]:
# temporary stuffs for latter (not needed)
# Using Beautiful soup to extract contents of all Paris arrondissement
soup_Paris_01 = BeautifulSoup(url_Paris_01, "html.parser")
soup_Paris_02 = BeautifulSoup(url_Paris_02, "html.parser")
soup_Paris_03 = BeautifulSoup(url_Paris_03, "html.parser")
soup_Paris_04 = BeautifulSoup(url_Paris_04, "html.parser")
soup_Paris_05 = BeautifulSoup(url_Paris_05, "html.parser")
soup_Paris_06 = BeautifulSoup(url_Paris_06, "html.parser")
soup_Paris_07 = BeautifulSoup(url_Paris_07, "html.parser")
soup_Paris_08 = BeautifulSoup(url_Paris_08, "html.parser")
soup_Paris_09 = BeautifulSoup(url_Paris_09, "html.parser")
soup_Paris_10 = BeautifulSoup(url_Paris_10, "html.parser")
soup_Paris_11 = BeautifulSoup(url_Paris_11, "html.parser")
soup_Paris_12 = BeautifulSoup(url_Paris_12, "html.parser")
soup_Paris_13 = BeautifulSoup(url_Paris_13, "html.parser")
soup_Paris_14 = BeautifulSoup(url_Paris_14, "html.parser")
soup_Paris_15 = BeautifulSoup(url_Paris_15, "html.parser")
soup_Paris_16 = BeautifulSoup(url_Paris_16, "html.parser")
soup_Paris_17 = BeautifulSoup(url_Paris_17, "html.parser")
soup_Paris_18 = BeautifulSoup(url_Paris_18, "html.parser")
soup_Paris_19 = BeautifulSoup(url_Paris_19, "html.parser")
soup_Paris_20 = BeautifulSoup(url_Paris_20, "html.parser")


In [61]:
# temporary stuffs for latter (not needed)
soup_Paris = [soup_Paris_01, soup_Paris_02, soup_Paris_03, soup_Paris_04, soup_Paris_05, soup_Paris_06, soup_Paris_07, soup_Paris_08, soup_Paris_09, soup_Paris_10, soup_Paris_11, soup_Paris_12,soup_Paris_13,soup_Paris_14,soup_Paris_15,soup_Paris_16,soup_Paris_17,soup_Paris_18,soup_Paris_19,soup_Paris_20]
# soup_Paris

In [62]:
# temporary stuffs for latter (not needed)
# list all childeren elements of soup_Paris_xx
children_Paris_01 = list(soup_Paris_01.children)
children_Paris_02 = list(soup_Paris_02.children)
children_Paris_03 = list(soup_Paris_03.children)
children_Paris_04 = list(soup_Paris_04.children)
children_Paris_05 = list(soup_Paris_05.children)
children_Paris_06 = list(soup_Paris_06.children)
children_Paris_07 = list(soup_Paris_07.children)
children_Paris_08 = list(soup_Paris_08.children)
children_Paris_09 = list(soup_Paris_09.children)
children_Paris_10 = list(soup_Paris_10.children)
children_Paris_19 = list(soup_Paris_11.children)
children_Paris_11 = list(soup_Paris_12.children)
children_Paris_12 = list(soup_Paris_13.children)
children_Paris_13 = list(soup_Paris_14.children)
children_Paris_14 = list(soup_Paris_15.children)
children_Paris_15 = list(soup_Paris_16.children)
children_Paris_16 = list(soup_Paris_17.children)
children_Paris_17 = list(soup_Paris_18.children)
children_Paris_18 = list(soup_Paris_19.children)
children_Paris_20 = list(soup_Paris_20.children)


In [63]:
# temporary stuffs for latter (not needed)
children_Paris = [children_Paris_01, children_Paris_02, children_Paris_03, children_Paris_04, children_Paris_05, children_Paris_06, children_Paris_07, children_Paris_08, children_Paris_09, children_Paris_10, children_Paris_11, children_Paris_12,children_Paris_13,children_Paris_14,children_Paris_15,children_Paris_16,children_Paris_17,children_Paris_18,children_Paris_19,children_Paris_20]
# children_Paris

In [64]:
# temporary stuffs for latter (not needed)
# works!!
for headline in soup_Paris_01.find_all("span", {"class": "mw-headline"}):
    print(headline.text)


Geography
Demography
Historical population
Immigration
Quarters
Economy
Education
Map
Cityscape
Places of interest
Bridges
Streets and squares
See also
References
External links


In [106]:
# temporary stuffs for latter (not needed)
# get all the links in the page along with its attributes, such as href, title, and its inner Text. (works!!)
# how to do it for all arrondissement??
# for link in soup_Paris_01.find_all("a"):
    print("Inner Text: {}".format(link.text))
    print("Title: {}".format(link.get("title")))
    print("href: {}".format(link.get("href")))

IndentationError: unexpected indent (<ipython-input-106-428185bdaf12>, line 5)

In [66]:
# placeofinterset = soup_Paris_01.find("span",id="Places_of_interest").find_next('ul')
# for place in placeofinterset.find_all('a'):
#    print(place['title']) #This will give you title
    #print(place.text) #This will give you text

In [67]:
# temporary stuffs for latter (not needed)
# soup_Paris_04.find_all('li', {'class': None}, {'id':'Places_of_interest'})

In [68]:
# temporary stuffs for latter (not needed)
# for li in cont_Paris_01:
#     content.append(li.getText().split('\n')[0])
# print(cont_Paris_01)

In [69]:
# temporary stuffs for latter (not needed)
# do smthg like
# Create Pandas Dataframe and export data into csv.
# df_bs = pd.DataFrame(row_list,columns=['City','Country','Notes'])
# df_bs.set_index('Country',inplace=True)
# df_bs.to_csv('beautifulsoup.csv')


In [70]:
# we resume here

## Foursquare API

In [71]:
# FourSquare Login
CLIENT_ID = 'RU04DQE0VHYJ0KWDVSX324MAPC43P0PT0ADKYYRAADXRIE4Q' # Foursquare ID
CLIENT_SECRET = 'QXFOBOGOIZRPOCQG5WCYMDZJKT4K4G0ERKJLR1YU5IGETJYM' # my Foursquare Secret
VERSION = '20191118' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

In [72]:
# function to retrieve nearby avenues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category' ]
    
    return(nearby_venues)

In [73]:
# Run the function, getNearbyVenues on each neighborhood and create a new dataframe called Paris_venues
Paris_venues = getNearbyVenues(names = paris_et_arrondissements_df['C_AR'],
                                   latitudes=paris_et_arrondissements_df['Latitude'],
                                   longitudes=paris_et_arrondissements_df['Longitude']
                                    )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [74]:
print(Paris_venues.shape)
Paris_venues.head(5)

(1361, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1,48.8625627018,2.33644336205,Musée du Louvre,48.860847,2.336440,Art Museum
1,1,48.8625627018,2.33644336205,Palais Royal,48.863236,2.337127,Historic Site
2,1,48.8625627018,2.33644336205,Comédie-Française,48.863088,2.336612,Theater
3,1,48.8625627018,2.33644336205,Cour Napoléon,48.861172,2.335088,Plaza
4,1,48.8625627018,2.33644336205,Place du Palais Royal,48.862523,2.336688,Plaza


In [76]:
# Number of venues per Districts
Paris_venues.groupby('District').count().sort_values('District')

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
1,100,100,100,100,100,100
2,100,100,100,100,100,100
3,100,100,100,100,100,100
4,100,100,100,100,100,100
5,96,96,96,96,96,96
6,92,92,92,92,92,92
7,100,100,100,100,100,100
8,64,64,64,64,64,64
9,100,100,100,100,100,100


In [77]:
# Number of unique categories
print('The number of unique categories is {}.'.format(len(Paris_venues['Venue Category'].unique())))

The number of unique categories is 201.


In [78]:
# Analyzing Paris Arrondissements (Districts)
# one hot encoding
Paris_onehot = pd.get_dummies(Paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Paris_onehot['District'] = Paris_venues['District'] 

# move district column to the first column
cols = list(Paris_onehot.columns.values)
cols.pop(cols.index('District'))
Paris_onehot = Paris_onehot[['District']+cols]

# rename Neighborhood for Districts so that future merge works
Paris_onehot.rename(columns = {'District': 'District'}, inplace = True)


,District,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [81]:
Paris_onehot.shape

(1361, 202)

In [79]:
Paris_onehot.describe()

,District,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
count,1361.000000,1361.000000,1361.000000,1361.00000,1361.00000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.00000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.00000,1361.000000,1361.000000,1361.00000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.00000,1361.00000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.00000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.00000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.00000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.000000,1361.00000,1361.00000,1361.000000,1361.000000,136

In [83]:
# Grouping rows by district and by the mean of the frequency of occurrence of each category
Paris_grouped = Paris_onehot.groupby('District').mean().reset_index()
Paris_grouped

,District,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,1,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.01,0.000000,0.00000,0.030000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000,0.000000,0.000000,0.00000,0.00,0.010000,0.00,0.010000,0.000000,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.070000,0.00000,0.000000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.00000,0.010000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.000000,0.01000,0.000000,0.00,0.00,0.00000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.090000,0.010000,0.020000,0.000000,0.00,0.00,0.01,0.000000,0.000000,0.00000,0.010000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.010000,0.070000,0.01,0.00,0.000000,0.000000,0.020000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0

In [85]:
Paris_grouped.shape

(20, 202)

In [92]:
# Printing districts along with the top 5 most common venues
num_top_venues = 5

for hood in Paris_grouped['District']:
    print("----" + str(hood) + "----")
    temp = Paris_grouped[Paris_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----1----
                 venue  freq
0    French Restaurant  0.09
1                Hotel  0.08
2                 Café  0.07
3  Japanese Restaurant  0.07
4                Plaza  0.06


----2----
                 venue  freq
0    French Restaurant  0.10
1             Wine Bar  0.06
2         Cocktail Bar  0.05
3   Italian Restaurant  0.04
4  Japanese Restaurant  0.04


----3----
               venue  freq
0  French Restaurant  0.08
1       Burger Joint  0.04
2               Café  0.04
3             Bistro  0.04
4        Coffee Shop  0.04


----4----
               venue  freq
0  French Restaurant  0.16
1     Ice Cream Shop  0.06
2        Pastry Shop  0.04
3             Bakery  0.03
4              Hotel  0.03


----5----
                venue  freq
0   French Restaurant  0.12
1               Hotel  0.05
2                Café  0.05
3               Plaza  0.04
4  Italian Restaurant  0.04


----6----
                venue  freq
0   French Restaurant  0.10
1  Italian Restaurant  0.10
2     

In [93]:
# Function to define the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
# Display the top 10 venues of each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['District'] = Paris_grouped['District']

for ind in np.arange(Paris_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,French Restaurant,Hotel,Café,Japanese Restaurant,Plaza,Coffee Shop,Bar,Udon Restaurant,Bakery,Art Museum
1,2,French Restaurant,Wine Bar,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Bistro,Restaurant,Bakery,Hotel,Thai Restaurant
2,3,French Restaurant,Café,Coffee Shop,Bistro,Burger Joint,Wine Bar,Sandwich Place,Moroccan Restaurant,Bakery,Italian Restaurant
3,4,French Restaurant,Ice Cream Shop,Pastry Shop,Hotel,Wine Bar,Pedestrian Plaza,Art Gallery,Bakery,Plaza,Thai Restaurant
4,5,French Restaurant,Café,Hotel,Science Museum,Italian Restaurant,Bar,Plaza,Creperie,Coffee Shop,Wine Bar
5,6,Italian Restaurant,French Restaurant,Bakery,Wine Bar,Bistro,Ice Cream Shop,Hotel,Pastry Shop,Bookstore,Athletics & Sports
6,7,Hotel,French Restaurant,Café,Italian Restaurant,Plaza,History Museum,Art Museum,Cocktail Bar,Ice Cream Shop,Garden
7,8,French Restaurant,Hotel,Bakery,Art Gallery,Japanese Restaurant,Thai Restaurant,Cocktail Bar,Theater,Mediterranean Restaurant,Bistro
8,9,French Restaurant,Hotel,Cocktail Bar,Bistro,Bakery,Wine Bar,Japanese Restaurant,Lounge,Coffee Shop,Sandwich Place
9,10,French Restaurant,Hotel,Bistro,Coffee Shop,Indian Restaurant,Café,Japanese Restaurant,Pizza Place,Bakery,Italian Restaurant


In [110]:
# dataframe for clusters and top 10 venues for each district
# Clustering Districts
Paris_merged = pd.concat([Paris_grouped,paris_et_arrondissements_df], axis=1)

# add clustering labels
Paris_merged['Cluster Labels'] = kmeans.labels_

# merge Paris_grouped with Paris_data to add latitude/longitude for each neighborhood
Paris_merged = Paris_merged.join(districts_venues_sorted.set_index('District'), on='District')

Paris_merged.head(21) # check the last columns!

,District,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Cluster Labels,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.01,0.000000,0.00000,0.030000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000,0.000000,0.000000,0.00000,0.00,0.010000,0.00,0.010000,0.000000,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.070000,0.00000,0.000000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.00000,0.010000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.000000,0.01000,0.000000,0.00,0.00,0.00000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.090000,0.010000,0.020000,0.000000,0.00,0.00,0.01,0.0

In [100]:
# Applying KMeans
kclusters = 5

Paris_grouped_clustering = Paris_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[1:10]

array([1, 1, 1, 1, 1, 4, 4, 1, 1])

In [111]:
# visualizing different arrondissements of Paris
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Paris_merged['Latitude'], Paris_merged['Longitude'], Paris_merged['District'], Paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



In [112]:
# Cluster 1 - Most common venues: Restaurants, Hotels and Bars
Paris_merged.loc[Paris_merged['Cluster Labels'] == 1, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]


,District,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Cluster Labels,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,0.00,0.00000,0.030000,0.000000,0.000000,0.000000,0.01,0.000000,0.00000,0.030000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.010000,0.0,0.000000,0.00000,0.00,0.010000,0.00,0.010000,0.000000,0.01,0.00,0.000000,0.00,0.00,0.00,0.0,0.070000,0.00000,0.0,0.000000,0.00,0.00,0.01,0.01,0.000000,0.00000,0.010000,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.02,0.00,0.00000,0.00,0.00,0.000000,0.01000,0.000000,0.00,0.00,0.00000,0.000000,0.020000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.090000,0.01,0.020000,0.000000,0.00,0.00,0.01,0.000000,0.00,0.00000,0.010000,0.000000,0.000000,0.000000,0.020000,0.0,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.01000

In [115]:
# Cluster 2 - Most common venues: Restaurants and parks (No Hotels in Top 10 venues)
Paris_merged.loc[Paris_merged['Cluster Labels'] == 2, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]


,District,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Cluster Labels,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.

In [116]:
# Cluster 3 - Most common venues: Nightclubs , Stations, Bars (No Hotels in Top 10 venues)
Paris_merged.loc[Paris_merged['Cluster Labels'] == 3, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,District,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Cluster Labels,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,16,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.153846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.153846,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [118]:
# Cluster 4 - Most common venues: Coffee shops, Restaurants, Grocery stores (No Hotels in Top 10 venues)
Paris_merged.loc[Paris_merged['Cluster Labels'] == 4, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,District,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Cluster Labels,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,7,0.01,0.000000,0.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,0.010000,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.010000,0.0,0.01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.060000,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.0,0.03000,0.02,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.010000,0.0,0.0,0.0,0.01,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.010000,0.0,0.180000,0.000000,0.02,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.010000,0.000000,0.000000,0.0,0.02,0.03,0.0,0.210000,0.000000,0.02,0.000000,0.0,0.0,0.000000,0.01,0.0,0.050000,0.020000,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.010000,0.000

Conclusions:
Cluster 1: From all 20 districts: The most popular are French restaurants with the exception of 18th (bar), 18th (bar), 20th districts (Pizza) and 6th (Italian). I believe that bars should have shown up in the 3rd, 6th or 10th and 11th districts.
The 2nd most popular choice is between hotel and French restaurants.

Cluster 2: From only 1 district: 12th arrondissement: The most popular choice is the zoo exhibit and the 2nd is the park. It is quite evident as there is Zoo de Vincennes and Park de Vincennes which are quite popular.

Cluster 3: From only 1 district: It is the 15th district and Foresquare tags it as 'lake' being most popular but realistically, most people go there to see the eiffel tower and take a river cruise on the river, Seine. The next popular is Pizza, which is quite obvious after some rigorous sightseeing.

Cluster 4: From 4 districts: Mostly for French restaurant and Hotel (7, 8, 17 arrondissements: These are quite touristic quartiers of Paris and perhaps the samples taken into account accounts for this bias (if FourSquare users are also rich). The 14th arrondissement is popular among the young and budget travellers.

Summary: Paris being a very touristic city, the results are bound to be more biased towards hotels and French restaurants. A more detailed study is necessary to find popular venues apart from these.